In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
from mapping import ResourceMapper
from graph import *
from sparql import SPAQLConverter

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
config = config_read('../')
rm = ResourceMapper(config)
cgg = ConceptualGraphGenerator(config)
qgg = QueryGraphGenerator(config)
sc = SPAQLConverter(config)

# 2-1. Query Partitioning

In [4]:
# user_input = '가수 태연의 첫 데뷔곡은?'
# user_input = '뱅뱅뱅 부른 음악그룹의 소속사는?'
user_input = '용감한 형제가 작곡한 노래의 장르는?'
# user_input = '작곡가이며 가수인 사람'
# user_input = '장르가 발라드인 노래'
# user_input = '가수 버벌진트가 피처링한 노래'
# user_input = '태연이 처음 부른 노래는?'
# user_input = '소녀시대의 리더는?'
# user_input = '제니의 소속 그룹'
# user_input = '레인을 작곡한 사람'
# user_input = '국적이 뉴질랜드 사람은?'
# user_input = '세월이 가면의 가수는?'
# user_input = 'AOA 부른 발라드 노래는?'
query_terms = rm.tokenize_query(user_input)
query_terms

['용감한', '형제', '작곡', '노래', '장르']

In [5]:
query_patitions = rm.partition_query(query_terms)
query_patitions

[['용감한 형제 작곡 노래 장르'],
 ['용감한', '형제 작곡 노래 장르'],
 ['용감한 형제', '작곡 노래 장르'],
 ['용감한 형제 작곡', '노래 장르'],
 ['용감한 형제 작곡 노래', '장르'],
 ['용감한', '형제', '작곡 노래 장르'],
 ['용감한', '형제 작곡', '노래 장르'],
 ['용감한', '형제 작곡 노래', '장르'],
 ['용감한 형제', '작곡', '노래 장르'],
 ['용감한 형제', '작곡 노래', '장르'],
 ['용감한 형제 작곡', '노래', '장르'],
 ['용감한', '형제', '작곡', '노래 장르'],
 ['용감한', '형제', '작곡 노래', '장르'],
 ['용감한', '형제 작곡', '노래', '장르'],
 ['용감한 형제', '작곡', '노래', '장르'],
 ['용감한', '형제', '작곡', '노래', '장르']]

# 2-2. Partition Scoring

In [6]:
p_star, score_list = rm.score_partition(query_patitions)
p_star

['용감한 형제', '작곡', '노래', '장르']

# 2-3. Generation of resource combination

In [7]:
resource_combinations = rm.combinate_resource(p_star)
resource_combinations

[('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track', 'skmo:Genre'),
 ('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track', 'skmo:hasGenre')]

# 3. Conceptual graph generation

In [8]:
conceptual_graph = cgg.generate_conceptual_graph(resource_combinations)
conceptual_graph

[[('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track'),
  ('mo:Track', 'Any P', 'skmo:Genre')],
 [('skmo:용감한_형제', 'skmo:isComposedBy', 'skmo:Genre'),
  ('mo:Track', 'Any P', 'skmo:Genre')],
 [('skmo:용감한_형제', 'skmo:isComposedBy', 'mo:Track'),
  ('mo:Track', 'skmo:hasGenre', 'owl:Thing')],
 [('skmo:용감한_형제', 'skmo:isComposedBy', 'owl:Thing'),
  ('mo:Track', 'skmo:hasGenre', 'owl:Thing')],
 [('skmo:용감한_형제', 'skmo:hasGenre', 'owl:Thing'),
  ('mo:Track', 'skmo:hasGenre', 'owl:Thing')]]

# 4. Query graph generation

In [9]:
shortest_path_dict = qgg.search_at_tbox_level(conceptual_graph)
shortest_path_dict

{('skmo:용감한_형제',
  'skmo:isComposedBy',
  'mo:Track'): [(0.7517241379310344,
   [('mo:Track', 'skmo:isComposedBy', 'foaf:Person(skmo:용감한_형제)')])],
 ('mo:Track',
  'Any P',
  'skmo:Genre'): [(0.0, [('mo:Track', 'skmo:hasGenre', 'skmo:Genre')])],
 ('skmo:용감한_형제', 'skmo:isComposedBy', 'skmo:Genre'): [],
 ('mo:Track',
  'skmo:hasGenre',
  'owl:Thing'): [(0.0,
   [('mo:Track', 'skmo:hasGenre', 'skmo:Genre'),
    ('skmo:Genre', 'rdfs:subClassOf', 'owl:Thing')])],
 ('skmo:용감한_형제',
  'skmo:isComposedBy',
  'owl:Thing'): [(0.8387096774193549,
   [('foaf:Person(skmo:용감한_형제)', 'skmo:debutTrack', 'mo:Track'),
    ('mo:Track', 'skmo:isComposedBy', 'skmo:MusicArtist'),
    ('skmo:MusicArtist', 'rdfs:subClassOf', 'owl:Thing')]), (0.5,
   [('foaf:Person(skmo:용감한_형제)', 'skmo:debutTrack', 'mo:Track'),
    ('mo:Track', 'skmo:isComposedBy', 'foaf:Organization'),
    ('foaf:Organization', 'rdfs:subClassOf', 'owl:Thing')])],
 ('skmo:용감한_형제',
  'skmo:hasGenre',
  'owl:Thing'): [(0.0,
   [('foaf:Person(skmo:용

In [10]:
query_graph = qgg.generate_query_graph(conceptual_graph, shortest_path_dict)
query_graph

[(0.6241379310344828,
  [('mo:Track', 'skmo:isComposedBy', 'foaf:Person(skmo:용감한_형제)'),
   ('mo:Track', 'skmo:hasGenre', 'skmo:Genre')]),
 (0.41609195402298854,
  [('mo:Track', 'skmo:isComposedBy', 'foaf:Person(skmo:용감한_형제)'),
   ('mo:Track', 'skmo:hasGenre', 'skmo:Genre'),
   ('skmo:Genre', 'rdfs:subClassOf', 'owl:Thing')]),
 (0.23225806451612901,
  [('foaf:Person(skmo:용감한_형제)', 'skmo:debutTrack', 'mo:Track'),
   ('mo:Track', 'skmo:isComposedBy', 'skmo:MusicArtist'),
   ('skmo:MusicArtist', 'rdfs:subClassOf', 'owl:Thing'),
   ('mo:Track', 'skmo:hasGenre', 'skmo:Genre'),
   ('skmo:Genre', 'rdfs:subClassOf', 'owl:Thing')]),
 (0.3,
  [('foaf:Person(skmo:용감한_형제)', 'skmo:debutTrack', 'mo:Track'),
   ('mo:Track', 'skmo:isComposedBy', 'foaf:Organization'),
   ('foaf:Organization', 'rdfs:subClassOf', 'owl:Thing'),
   ('mo:Track', 'skmo:hasGenre', 'skmo:Genre'),
   ('skmo:Genre', 'rdfs:subClassOf', 'owl:Thing')]),
 (0.4,
  [('foaf:Person(skmo:용감한_형제)', 'skmo:debutTrack', 'mo:Track'),
   ('mo:T

# 5. SPARQL Conversion

In [11]:
sc.process(query_graph)

k-pop  R&B  hip_hop  

In [12]:
final_query_graph = sc.select_query_graph(query_graph)
final_query_graph

[('mo:Track', 'skmo:isComposedBy', 'foaf:Person(skmo:용감한_형제)'),
 ('mo:Track', 'skmo:hasGenre', 'skmo:Genre')]

In [13]:
relation_triples = sc.extract_relation_triples(final_query_graph)
relation_triples

[('mo:Track', 'skmo:isComposedBy', 'foaf:Person(skmo:용감한_형제)'),
 ('mo:Track', 'skmo:hasGenre', 'skmo:Genre')]

In [14]:
print('<USER Query>')
print(user_input)
print()


print('<SPARQL Query>')
knows_query = sc.generate_query(relation_triples)
print(knows_query)


print('\n<Answer>')
result = sc.excute_query(knows_query)
sc.print_result(result)

<USER Query>
용감한 형제가 작곡한 노래의 장르는?

<SPARQL Query>
SELECT DISTINCT ?target 
WHERE
{
    ?x0 skmo:isComposedBy skmo:용감한_형제; skmo:hasGenre ?target
}

<Answer>
k-pop  R&B  hip_hop  